Mémoire d'Anne-Lou Pichet

# Chargement des bibliothèques et fonctions. 

In [ ]:
import pandas as pd
import numpy as np
import re
import spacy
#!python -m spacy download fr_core_news_lg

nlp = spacy.load("fr_core_news_lg")

## Fonctions

In [ ]:
# Listes de noms par type de carrière
carriere_loisir_socio = ['Agathe','Alya', 'Jules', 'Salomé', 'Alice ']
carriere_semi_professionnelle = ['Clément', 'Claire ', 'Amélie', 'Capucine', 'Hélène', 'Christine ', 'Laureline', 'Elisabeth', 'Jade']
carriere_milieu_socio = ['Aline', 'Arthur', 'Emma', 'Ariane', 'Maude']
# Fonction pour déterminer le type de carrière
def get_type_carriere(nom):
    if nom in carriere_loisir_socio:
        return 'loisir'
    elif nom in carriere_semi_professionnelle:
        return 'semi_professionnelle'
    elif nom in carriere_milieu_socio:
        return 'milieu'
    else:
        return 'inconnu'

In [ ]:
# Fonction pour extraire et classifier les hashtags
def analyser_hashtags(post):
    hashtags = re.findall(r"#(\w+)", str(post))
    total_hashtags = len(hashtags)
    return {
        "Contient_Hashtags": total_hashtags > 0,
        "Nombre_Hashtags": total_hashtags,
        "Liste_Hashtags": hashtags
    }


In [ ]:
# Fonction pour extraire et classifier les hashtags dans les posts (commençant par un #)
def analyser_hashtags(post):
    hashtags = re.findall(r"#(\w+)", str(post))
    total_hashtags = len(hashtags)
    return {
        "Contient_Hashtags": total_hashtags > 0,
        "Nombre_Hashtags": total_hashtags,  
        "Liste_Hashtags": hashtags,  
    }

In [ ]:
# Fonction pour extraire et classifier les mentions dans les posts (commençant par un @)
def analyser_mentions(post):
    mentions = re.findall(r"@(\w+)", str(post))
    total_mentions = len(mentions)
    mentions_auteurs = [m for m in mentions if m.startswith("A")]  # Mentions d'auteurs
    mentions_maisons = [m for m in mentions if m.startswith("ME")]  # Mentions de maisons d'édition
    mentions_pairs = [m for m in mentions if m.startswith("Pa")]  # Mentions de pairs
    mentions_autres = [m for m in mentions if not (m.startswith("A") or m.startswith("ME") or m.startswith("Pa"))]  # Mentions autres
    return {
        "Contient_Mentions": total_mentions > 0,  
        "Nombre_Mentions": total_mentions,  
        "Mentions_Auteurs": len(mentions_auteurs),  
        "Mentions_Maisons": len(mentions_maisons),  
        "Mentions_Autres": len(mentions_autres),  
        "Mentions_Pairs": len(mentions_pairs),  
        "Contient_Mentions_Pairs": len(mentions_pairs) > 0,  
        "Contient_Mentions_Auteurs": len(mentions_auteurs) > 0,
        "Contient_Mentions_Autres": len(mentions_autres) > 0,
        "Contient_Mentions_Maisons": len(mentions_maisons) > 0
    }

# Chargement des données et prétraitements de celles-ci  

Ce code a pour but de transformer la structure de mes données pour pouvoir, par la suite, construire des statistiques à leur propos.

In [ ]:
data_brutes = pd.read_excel("Posts_Anon.xlsx") # fichier d'origine des données à transformer 
images_brutes = pd.read_excel("Images_Posts.xlsx", engine='openpyxl') # fichier contenant les données des images

### Transformation des données brutes

In [ ]:
# Liste pour stocker les données transformées
format_propre = []

# Pour chaque auteur dans le DataFrame, on parcourt les colonnes et on remet en forme les données
for _, row in data_brutes.iterrows():
    for col in data_brutes.columns:
        match = re.match(r"Contenu_([A-Za-z0-9]+)_Post", col)  # On extrait le code du post (ex: F1, L2, A1E3...) 
        if match:  
            code_post = match.group(1)  # On récupère le code du post
            
            # Récupérer les valeurs associées aux autres attributs
            contenu = row[f"Contenu_{code_post}_Post"]
            type_post = row.get(f"Type_{code_post}_Post", None)
            annexes = row.get(f"Annexes_{code_post}_Post", None)
            date_post = row.get(f"Date_{code_post}_Post", None)
            
            code_propre = None
            nombre_propre = None
            code_match = re.match(r"^(.+?)(?=\d+$)", code_post)  # Tout sauf les derniers chiffres
            nombre_match = re.search(r"(\d+)$", code_post)       # Les derniers chiffres uniquement
            code_propre = code_match.group(1)
            nombre_propre = nombre_match.group(1)
            nombre_propre = float(nombre_propre)

            # Si un contenu ou une date existe, ajouter une nouvelle ligne
            if pd.notna(date_post) or pd.notna(contenu):
                format_propre.append({
                    "Nom_d_emprunt": row["Nom d'emprunt"],
                    "Contenu_Post": contenu,
                    "Type_Post": type_post,
                    "Code_Post": code_post, 
                    "Code": code_propre,
                    "Nombre": nombre_propre,
                    "Annexes_Post": annexes,
                    "Date_Post": date_post
                })
             
# Créer un DataFrame final
data_propre = pd.DataFrame(format_propre)


### Nettoyage Posts

In [ ]:
# Dictionnaire de correspondance pour corriger les erreurs
corrections = {
    "Vidéos": "Vidéo",
    "Vdéo": "Vidéo",
    "Videos": "Vidéo",
    "DIvers": "Divers",
    "Dievrs": "Divers",
    "Dvers": "Divers",
    "Recap": "Récap",
    "chronique": "Chronique",
    "recommandations": "Recommandations",
    "Recommendations": "Recommandations",
    "Auteur": "Professionnel",  # Si "Auteur" désigne un post pro
    "Extrait": "Extrait",
    "Citation": "Citation",
    "Podcast": "Audio",
    "Haul": "Haul",
    "Challenge": "Challenge",
    "Présentation": "Présentation",
    "Bilan": "Bilan",
    "Récap": "Récap",
}

# Supprimer les valeurs clairement incorrectes
valeurs_a_supprimer = {'', 'p', ']', 'www.instagram.com', 'https:', 'CthTGTTIj7u'}

# Fonction pour nettoyer les types de post
def clean_type_post(t):
    if pd.isna(t):
        return None
    cleaned_values = {corrections.get(x.strip(), x.strip()) for x in t.split('/')}
    cleaned_values.difference_update(valeurs_a_supprimer)  
    cleaned_values = {re.sub(r'\[.*?\]|https?:\/\/\S+', '', x) for x in cleaned_values}
    cleaned_values.discard('')
    return '/'.join(sorted(cleaned_values)) if cleaned_values else None  

# Appliquer le nettoyage
data_propre['Type_Post'] = data_propre['Type_Post'].apply(clean_type_post)

# Afficher les nouvelles valeurs uniques
print(data_propre['Type_Post'].unique())

### Déterminer la nature des posts

In [ ]:
def determine_post_type(row):
    type_post = row['Type_Post']
    
    if pd.isna(type_post):
        return None  # Aucun type si NaN
    
    types = set(type_post.split('/'))  # Séparer les types multiples

    # On garde toutes les catégories valides
    categories = set()
    if 'Vidéo' in types:
        categories.add('Vidéo')
    if 'Audio' in types:
        categories.add('Audio')
    if pd.notna(row['Contenu_Post']):
        categories.add('Écrit')

    return '/'.join(sorted(categories)) if categories else None  # Trier pour homogénéité

# Appliquer la fonction mise à jour
data_propre['Type_Post'] = data_propre.apply(determine_post_type, axis=1)

# Créer les colonnes binaires
data_propre['N_Video'] = data_propre['Type_Post'].str.contains('Vidéo', na=False).astype(int)
data_propre['N_Audio'] = data_propre['Type_Post'].str.contains('Audio', na=False).astype(int)
data_propre['N_Ecrit'] = data_propre['Type_Post'].str.contains('Écrit', na=False).astype(int)

data_propre.drop(columns=['Type_Post'], inplace=True)

### Merge avec les données des images

In [ ]:
# ajout d'un index commun
data_propre['index'] = data_propre.groupby(['Nom_d_emprunt', 'Date_Post']).cumcount()
images_brutes['index'] = images_brutes.groupby(['Nom_d_emprunt', 'Date_Post']).cumcount()

# conversion des colonnes spécifiques en numérique
images_brutes[['Plusieurs_Images', 'Personne', 'PostProduction', 'Mots']] = images_brutes[['Plusieurs_Images', 'Personne', 'PostProduction', 'Mots']].apply(pd.to_numeric, errors='coerce')

# Fusion des données
base_complete = pd.merge(data_propre, images_brutes, on=['Nom_d_emprunt', 'Date_Post', 'index'], how='left')


# Save the merged_data to an Excel file
#base_complete.to_excel("Base_Posts_Anon.xlsx", index=False)

## Données traitées pour les statistiques

In [ ]:
data = pd.read_excel("Base_Posts_Anon.xlsx")

## Gestion des périodes et des identifiants.

In [ ]:
# On tranforme les périodes pour qu'elles soient plus lisibles
data["Date_Post"] = pd.to_datetime(data["Date_Post"], errors="coerce")
data["Periode"] = data["Code"].str.extract(r"^([A-Za-z]+[0-9]*)") 

recodage_periodes = {"F": 1, "A1": 2, "A2": 3, "L": 4} # Dictionnaire de recodage des périodes
# On gére les posts épinglés
data["Periode"] = data["Periode"].apply(lambda x: recodage_periodes.get(x, "E"))

# On ajoute un Identifiant pour chaque post
data["ID_Post"] = range(1, len(data) + 1)
data.set_index("ID_Post", inplace=True)

## Ajout des types de carrières 

In [ ]:
data['type_carriere'] = data['Nom_d_emprunt'].apply(get_type_carriere)

## Réattribuer les posts épinglés

In [ ]:
data["Date_Post"] = pd.to_datetime(data["Date_Post"], errors="coerce")
data["Période_Bis"] = data["Periode"] 
data = data.sort_values(by=["Nom_d_emprunt", "Periode", "Date_Post"])

In [ ]:
# Réaffectation des posts "E"
for auteur in data["Nom_d_emprunt"].unique():
    auteur_data = data[data["Nom_d_emprunt"] == auteur].copy()

    posts_E = auteur_data[auteur_data["Periode"] == "E"]
    autres_periodes = auteur_data[auteur_data["Periode"] != "E"]

    for periode in autres_periodes["Periode"].unique():
        periode_data = autres_periodes[autres_periodes["Periode"] == periode]
        if periode_data["Date_Post"].isnull().all():
            continue
        min_date = periode_data["Date_Post"].min()
        max_date = periode_data["Date_Post"].max()

        for idx_E, row_E in posts_E.iterrows():
            date_E = row_E["Date_Post"]
            if pd.notnull(date_E) and min_date <= date_E <= max_date:
                data.at[idx_E, "Période_Bis"] = periode

In [ ]:
# Création du tableau des plages par période
plages_periode = []

for auteur in data["Nom_d_emprunt"].unique():
    auteur_data = data[(data["Nom_d_emprunt"] == auteur) & (data["Periode"] != "E")]

    for periode in auteur_data["Periode"].unique():
        periode_data = auteur_data[auteur_data["Periode"] == periode]
        if periode_data["Date_Post"].isnull().all():
            continue

        plages_periode.append({
            "Auteur": auteur,
            "Periode": periode,
            "Date_Début": periode_data["Date_Post"].min(),
            "Date_Fin": periode_data["Date_Post"].max()
        })

plages = pd.DataFrame(plages_periode)

In [ ]:
# Vérification des rattachements des posts épinglés ("E")
posts_E_originaux = data[data["Periode"] == "E"]

verifications = []

for idx, row in posts_E_originaux.iterrows():
    auteur = row["Nom_d_emprunt"]
    date_post = row["Date_Post"]
    periode_bis = row["Période_Bis"]

    valide = False

    auteur_data = data[(data["Nom_d_emprunt"] == auteur) & (data["Periode"] != "E")]
    for periode in auteur_data["Periode"].unique():
        periode_data = auteur_data[auteur_data["Periode"] == periode]
        if periode_data["Date_Post"].isnull().all():
            continue
        min_date = periode_data["Date_Post"].min()
        max_date = periode_data["Date_Post"].max()

        if pd.notnull(date_post) and min_date <= date_post <= max_date:
            valide = (periode_bis == periode)
            break

    verifications.append({
        "Index": idx,
        "Auteur": auteur,
        "Date_Post": date_post,
        "Période_Bis": periode_bis,
        "Date_dans_une_plage_connue": valide
    })

verifications = pd.DataFrame(verifications)

In [ ]:
# On vérifie les dates des posts épinglés par rapport aux plages connues
verifs = verifications.merge(plages, how="left",
                                  left_on=["Auteur", "Période_Bis"],
                                  right_on=["Auteur", "Periode"])

verifs = verifs[[
    "Index", "Auteur", "Date_Post", "Période_Bis", "Date_dans_une_plage_connue",
    "Date_Début", "Date_Fin"
]].sort_values(by=["Auteur", "Date_Post"])


print("Nombre de posts 'E' au départ :", len(posts_E_originaux))
print("Nombre de posts 'E' encore non rattachés :", (data["Période_Bis"] == "E").sum())
print("\nRattachements incorrects ou non attribués :")
verifs[verifs["Date_dans_une_plage_connue"] == False] # Pour voir si il y a des erreurs

In [ ]:
# Vérification des périodes pour certains enquêtés pour vérifier les rattachements à la main 
enquetes = ['Amélie', 'Capucine', 'Christine', 'Claire', 'Emma', 'Jade', 'Laureline', 'Salomé']

for auteur in data["Nom_d_emprunt"].unique():
    if auteur in enquetes:
        auteur_data = data[(data["Nom_d_emprunt"] == auteur) & (data["Periode"] != "E")]
        print(f"\nPériodes pour l'auteur {auteur} :")
        for periode in auteur_data["Periode"].unique():
            periode_data = auteur_data[auteur_data["Periode"] == periode]
            if not periode_data["Date_Post"].isnull().all():
                min_date = periode_data["Date_Post"].min()
                max_date = periode_data["Date_Post"].max()
                print(f"Période {periode}: Début: {min_date}, Fin: {max_date}")

In [ ]:
data.to_excel("Base_Posts_Anon.xlsx") # Pour sauvegarder les modifications la première fois que le script est exécuté

# Création des indicateurs statistiques

## Médiane des posts

In [ ]:
# obtenir le nombre total et la médiane de posts par enquêté
posts_par_enquete = data.groupby('Nom_d_emprunt').size().reset_index(name='Nombre de Posts')
mediane_posts = posts_par_enquete['Nombre de Posts'].median()
mediane_posts 

## Fréquence de posts 

On analyse ici la fréquence à laquelle nos enquêtés postent sur Instagram à propos de leurs lectures. 
Pour cela, nous calculons la fréquence entre chaque posts d'une même période et pour un même enquêté. Nous croiserons par la suite les résulats avec plusieurs autres indicateurs.  

In [ ]:
data_freq = data.sort_values(by=["Nom_d_emprunt", "Période_Bis", "Date_Post"])

frequence = []

for auteur in data_freq["Nom_d_emprunt"].unique():
    auteur_data = data_freq[data_freq["Nom_d_emprunt"] == auteur]
    carriere = auteur_data["type_carriere"].unique()
    for periode in auteur_data["Période_Bis"].unique():
        periode_data = auteur_data[auteur_data["Période_Bis"] == periode]
        
        if periode_data["Date_Post"].isnull().all():
            frequence.append({
                "Nom_d_emprunt": auteur,
                "Période_Bis": periode,
                "Carrière": carriere,
                "Nombre_de_Posts": 0,
                "Intervalle_Moyen": None,
                "Intervalle_Minimum": None,
                "Intervalle_Maximum": None,
                "Ecart_Type_Intervalle": None
            })
            continue
        
        nombre_de_posts = len(periode_data)
        
        periode_data = periode_data.sort_values(by="Date_Post")
        periode_data["Intervalle_Posts"] = periode_data["Date_Post"].diff().dt.days
        
        periode_data = periode_data.dropna(subset=["Intervalle_Posts"])
        
        if not periode_data.empty:
            frequence.append({
                "Nom_d_emprunt": auteur,
                "Période_Bis": periode,
                "Carrière": carriere,
                "Nombre_de_Posts": nombre_de_posts, 
                "Intervalle_Moyen": periode_data["Intervalle_Posts"].mean(),
                "Intervalle_Minimum": periode_data["Intervalle_Posts"].min(),
                "Intervalle_Maximum": periode_data["Intervalle_Posts"].max(),
                "Intervalle_Mediane": periode_data["Intervalle_Posts"].median(),
                "Ecart_Type_Intervalle": periode_data["Intervalle_Posts"].std()
            })
        else:
            # Si les intervalles ne peuvent pas être calculés, on ajoute des valeurs par défaut
            frequence.append({
                "Nom_d_emprunt": auteur,
                "Période_Bis": periode,
                "Carrière": carriere,
                "Nombre_de_Posts": nombre_de_posts,
                "Intervalle_Moyen": None,
                "Intervalle_Minimum": None,
                "Intervalle_Maximum": None,
                "Intervalle_Mediane": None,
                "Ecart_Type_Intervalle": None
            })

frequence = pd.DataFrame(frequence)
frequence.rename(columns={"Période_Bis": "Periode",}, inplace=True)
frequence.head()



### Analyse des fréquences selon les carrières

On croise l'analyse des fréquences de posts avec les carrières. 
Comme nous avons un outlier impactant fortement les données des fréquences (Capucine), nous avons voulu voir la différence en l'enlevant. 

In [ ]:
freq_carriere = []

frequence["Carrière"] =frequence["Carrière"].apply(lambda x: str(x))
for carriere in frequence["Carrière"].unique():
    carriere_data = frequence[frequence["Carrière"] == carriere]

    # Suppression de l'outlier pour voir son effet sur la moyenne
    max_val = carriere_data['Intervalle_Moyen'].max()
    carriere_data_sans_outlier = carriere_data[carriere_data['Intervalle_Moyen'] != max_val]

    freq_carriere.append({
        "Carrière": carriere,
        "Nombre de posts": carriere_data['Nombre_de_Posts'].sum(),
        "Intervalle minimum": carriere_data['Intervalle_Minimum'].min(),
        "Intervalle maximum": carriere_data['Intervalle_Maximum'].max(),
        "Intervalle moyen": carriere_data['Intervalle_Moyen'].mean(),
        "Intervalle médian": carriere_data['Intervalle_Mediane'].median(),
        "Intervalle moyen (sans outlier)": carriere_data_sans_outlier['Intervalle_Moyen'].mean()
    })

freq_carriere = pd.DataFrame(freq_carriere)
freq_carriere

Nous allons maintenant chercher les statistiques de fréquences croisées avec les carrières et les périodes. 

In [ ]:
frequence.columns

In [ ]:
freq_carriere_periode = []
for carriere in frequence["Carrière"].unique():
    carriere_data = frequence[frequence["Carrière"] == carriere]

    # Boucle sur chaque période sauf les posts épinglés ('E')
    for periode in carriere_data["Periode"].unique():
        if periode == 'E':
            continue
        periode_data = carriere_data[carriere_data["Periode"] == periode]

        nb_posts = periode_data["Nombre_de_Posts"].sum()
        intervalle_min = periode_data["Intervalle_Minimum"].min()
        intervalle_max = periode_data["Intervalle_Maximum"].max()
        intervalle_moy = periode_data["Intervalle_Moyen"].mean()
        intervalle_med = periode_data["Intervalle_Mediane"].median()

        freq_carriere_periode.append({
            "Carrière": carriere,
            "Période": periode,
            "Nombre de posts": nb_posts,
            "Intervalle minimum": intervalle_min,
            "Intervalle maximum": intervalle_max,
            "Intervalle moyen": round(intervalle_moy, 2), 
            "Intervalle médian": round(intervalle_med, 2),
        })

freq_carriere_periode = pd.DataFrame(freq_carriere_periode)

freq_carriere_periode.sort_values(by=["Carrière", "Période"])

### Selon les périodes

On va ici croiser les statistiques de fréquence avec les périodes, pour identifier de potentielles variations dans le temps. Pour cela, on inclut des pondérations car tout les enquêtés n'ont pas le meme nombre de posts pour chaque période. 

In [ ]:
frequence_periode = pd.DataFrame(columns=["Période"])
for periode_bis in frequence["Période_Bis"].unique():
    periode_data = frequence[frequence["Période_Bis"] == periode_bis]
    # intervalle moyen
    intervalle_moyen_pondere = (periode_data["Intervalle_Moyen"] * periode_data["Nombre_de_Posts"]).sum() / periode_data["Nombre_de_Posts"].sum()
    # Intervalle minimum
    intervalle_minimum = periode_data["Intervalle_Minimum"].min()
    # Intervalle maximum
    intervalle_maximum = periode_data["Intervalle_Maximum"].max()
    # Intervalle median
    mediane_intervalle = periode_data["Intervalle_Moyen"].median()
    #ecart type pondéré
    ecart_type_pondere = (periode_data["Ecart_Type_Intervalle"] * periode_data["Nombre_de_Posts"]).sum() / periode_data["Nombre_de_Posts"].sum()
    # Nombre total de posts dans cette période
    nombre_posts = periode_data["Nombre_de_Posts"].sum()

    # Ajouter les résultats au tableau récapitulatif
    nouvelle_ligne = pd.DataFrame([{
        "Période": periode_bis,
       "Intervalle_Moyen": intervalle_moyen_pondere,
       "Médiane_Intervalle": mediane_intervalle,
        "Intervalle_Minimum": intervalle_minimum,
        "Intervalle_Maximum": intervalle_maximum,
        "Ecart_Type": ecart_type_pondere,
        "Intervalle_Mediane": mediane_intervalle,
        "Nombre_de_Posts": nombre_posts,

    }])

    frequence_periode = pd.concat([frequence_periode, nouvelle_ligne], ignore_index=True)

# Ajout des données pour toutes les périodes 

ligne_total = pd.DataFrame([{
    "Période": "Total",
    "Intervalle_Moyen": frequence_periode["Intervalle_Moyen"].mean(),
    "Médiane_Intervalle": frequence_periode["Médiane_Intervalle"].median(),
    "Intervalle_Minimum": frequence_periode["Intervalle_Minimum"].min(),
    "Intervalle_Maximum": frequence_periode["Intervalle_Maximum"].max(),
    "Ecart_Type": frequence_periode["Ecart_Type"].mean(),
    "Nombre_de_Posts": frequence_periode["Nombre_de_Posts"].sum(),
}])

frequence_periode = pd.concat([frequence_periode, ligne_total], ignore_index=True)

frequence_periode


## Longueur des posts et des annexes 

On va ici analyser le contenu des posts par leur longeur (à la fois le corps des posts mais aussi leurs éventuelles annexes). 

Nous avons pour cette analyse, tenté plusieurs choses : 
- un calcul de la longeur en mots et en caractères. On a finalement retenu ce dernier indicateurs pour nos analyses. 
- nous avons envisagé de faire une analyse plus poussée sur le langage utilisée dans les posts de nos enquêtés, avant de plutôt nous tourner vers des clusterings pour identifier des clusters de pratiques et d'enquêtés. Dans un travail suivant, il serait alors pertinent de poursuivre par une analyse TAL. 

In [ ]:
# Calculer la longueur des posts (en caractères et mots)
data["Contenu_Post"] = data["Contenu_Post"].astype(str)  # Gérer les NaN
data["Longueur_Mots"] = data["Contenu_Post"].apply(lambda x: len(x.split()))  # Longueur en mots
data["Longueur_Contenu"] = data["Contenu_Post"].apply(lambda x: len(str(x)) if pd.notna(x) else 0)

data["Presence_Annexes"] = data["Annexes_Post"].apply(lambda x: bool(pd.notna(x) and str(x).strip() and str(x).lower() != 'nan'))
# On identifie les annexes dans les posts avec [AP]
data["Annexes_Post"] = data["Annexes_Post"].astype(str)  # Convertir en chaînes de caractères
data["Longueur_Annexes"] = data["Annexes_Post"].str.replace(r"\[AP\]", "", regex=True).apply(len)
data["Nombre_Annexes"] = data["Annexes_Post"].apply(
    lambda x: len(str(x).split("[AP]")) if pd.notna(x) and str(x).strip() and str(x).lower() != 'nan' else 0
)

# Calculer la longueur totale (contenu + annexes)
data["Longueur_Totale"] = data["Longueur_Contenu"] + data["Longueur_Annexes"]


# Colonnes pour les caractéristiques NLP des posts et de leurs annexes
data["Entites_Nommees"] = None
data["Tags_POS"] = None
data["Vecteur_Norm"] = None
data["Entites_Nommees_Annexes"] = None
data["Tags_POS_Annexes"] = None
data["Vecteur_Norm_Annexes"] = None

# Analyse NLP sur chaque post et ses annexes
for i, row in data.iterrows():
    doc_post = nlp(row["Contenu_Post"])
    entites_post = [(ent.text, ent.label_) for ent in doc_post.ents]
    pos_tags_post = [(token.text, token.pos_) for token in doc_post]
    vecteur_norm_post = doc_post.vector_norm
       
    # On ajoute les résultats pour le corps du post
    data.at[i, "Entites_Nommees"] = str(entites_post)
    data.at[i, "Tags_POS"] = str(pos_tags_post)
    data.at[i, "Vecteur_Norm"] = vecteur_norm_post
    
    # Puis on fait de même pour les annexes
    annexes = row["Annexes_Post"].split("[AP]") if row["Annexes_Post"].strip() else []
    entites_annexes = []
    pos_tags_annexes = []
    vecteur_norm_annexes = 0
    
    if annexes:
        for annexe in annexes:
            annexe = annexe.strip()
            if annexe and annexe != "nan": 
                doc_annexe = nlp(annexe)
                entites_annexes.extend([(ent.text, ent.label_) for ent in doc_annexe.ents])
                pos_tags_annexes.extend([(token.text, token.pos_) for token in doc_annexe])
                vecteur_norm_annexes += doc_annexe.vector_norm 
    data.at[i, "Entites_Nommees_Annexes"] = str(entites_annexes)
    data.at[i, "Tags_POS_Annexes"] = str(pos_tags_annexes)
    data.at[i, "Vecteur_Norm_Annexes"] = vecteur_norm_annexes


### Analyse de la longeur des posts

In [ ]:
# Longueur totale (contenu + annexes)
longueur_totale_data = data.dropna(subset=["Longueur_Totale"])

#rajouter la précense des annexes en pourcentages
stats_globales_longueur = {
    "Type": "Global",
    "Nombre_de_Posts": len(longueur_totale_data),
    "Longueur_Moyenne": longueur_totale_data["Longueur_Totale"].mean(),
    "Longueur_Minimale": longueur_totale_data["Longueur_Totale"].min(),
    "Longueur_Maximale": longueur_totale_data["Longueur_Totale"].max(),
    "Longueur_Mediane": longueur_totale_data["Longueur_Totale"].median(),
    "Ecart_Type_Longueur": longueur_totale_data["Longueur_Totale"].std(),
    "Pourcentage_Annexes": (longueur_totale_data["Presence_Annexes"].sum() / len(longueur_totale_data)) * 100,
    "Nombres d'Annexes" : longueur_totale_data["Nombre_Annexes"].sum()
}

In [ ]:
periode_longueur = []

for periode in data["Periode"].dropna().unique():
    periode_data = data[data["Periode"] == periode].dropna(subset=["Longueur_Totale"])
    
    if not periode_data.empty:
        periode_longueur.append({
            "Type": f"Période: {periode}",
            "Nombre_de_Posts": len(periode_data),
            "Longueur_Moyenne": periode_data["Longueur_Totale"].mean(),
            "Longueur_Minimale": periode_data["Longueur_Totale"].min(),
            "Longueur_Maximale": periode_data["Longueur_Totale"].max(),
            "Longueur_Mediane": periode_data["Longueur_Totale"].median(),
            "Ecart_Type_Longueur": periode_data["Longueur_Totale"].std(),
            "Pourcentage_Annexes": (periode_data["Presence_Annexes"].sum() / len(periode_data)) * 100, 
            "Nombres d'Annexes" : periode_data["Nombre_Annexes"].sum()  

        })

In [ ]:
# On affiche les résultats globaux et par période
longueur = pd.DataFrame([stats_globales_longueur] + periode_longueur)

longueur.head()

In [ ]:

enquete_longueur = []

for (nom_enquete, periode), couple in data.groupby(["Nom_d_emprunt", "Periode"]):
    if not couple.empty:
        enquete_longueur.append({
            "Nom_d_emprunt": nom_enquete,
            "Periode": periode,
            "Nombre_de_Posts": len(couple),
            "Longueur_Moyenne": couple["Longueur_Totale"].mean(),
            "Longueur_Minimale": couple["Longueur_Totale"].min(),
            "Longueur_Maximale": couple["Longueur_Totale"].max(),
            "Ecart_Type_Longueur": couple["Longueur_Totale"].std(),
            "Longueur_Mediane": couple["Longueur_Totale"].median(),
            "Pourcentage_Annexes": (couple["Presence_Annexes"].sum() / len(couple)) * 100,
            "Nombres d'Annexes": couple["Nombre_Annexes"].sum()
        })
enquete_longueur = pd.DataFrame(enquete_longueur)
enquete_longueur = enquete_longueur.sort_values(by=["Nom_d_emprunt", "Periode"])
enquete_longueur

In [ ]:
longueur_carriere = []
for carriere in data["type_carriere"].unique():
    carriere_data = data[data["type_carriere"] == carriere].dropna(subset=["Longueur_Totale"])
    
    if not carriere_data.empty:
        longueur_carriere.append({
            "Type": f"Carrière: {carriere}",
            "Nombre_de_Posts": len(carriere_data),
            "Longueur_Moyenne": carriere_data["Longueur_Totale"].mean(),
            "Longueur_Minimale": carriere_data["Longueur_Totale"].min(),
            "Longueur_Maximale": carriere_data["Longueur_Totale"].max(),
            "Ecart_Type_Longueur": carriere_data["Longueur_Totale"].std(),
            "Longueur_Mediane": carriere_data["Longueur_Totale"].median(),
            "Pourcentage_Annexes": (carriere_data["Presence_Annexes"].sum() / len(carriere_data)) * 100,
            "Nombres d'Annexes": carriere_data["Nombre_Annexes"].sum()
        })

longueur_carriere = pd.DataFrame(longueur_carriere)
longueur_carriere

## Mentions dans les posts 

On va ici analyser l'usage que nos enquêtés font des mentions (d'auteurs, de maisons d'éditions, de leurs pairs...) à l'aide de @, au sein de leurs posts. 

In [ ]:
# Analyse pour contenu et annexes
for zone, serie in [("Contenu", data["Contenu_Post"]), ("Annexes", data["Annexes_Post"])]:
    mentions = serie.apply(analyser_mentions)
    for key in mentions.iloc[0].keys():  # récupère toutes les clés du dict
        data[f"{key}_{zone}"] = mentions.apply(lambda x: x[key])

# Colonnes totales (booléens)
data["Contient_Mention"] = data["Contient_Mentions_Contenu"] | data["Contient_Mentions_Annexes"]
data["Contient_Mentions_Pairs_Total"] = data["Contient_Mentions_Pairs_Contenu"] | data["Contient_Mentions_Pairs_Annexes"]
data["Contient_Mentions_Auteurs_Total"] = data["Contient_Mentions_Auteurs_Contenu"] | data["Contient_Mentions_Auteurs_Annexes"]
data["Contient_Mentions_Autres_Total"] = data["Contient_Mentions_Autres_Contenu"] | data["Contient_Mentions_Autres_Annexes"]
data["Contient_Mentions_Maisons_Total"] = data["Contient_Mentions_Maisons_Contenu"] | data["Contient_Mentions_Maisons_Annexes"]

# Colonnes totales (nombres)
data["Nombre_Mentions_Total"] = data["Nombre_Mentions_Contenu"] + data["Nombre_Mentions_Annexes"]
data["Mentions_Auteurs_Total"] = data["Mentions_Auteurs_Contenu"] + data["Mentions_Auteurs_Annexes"]
data["Mentions_Maisons_Total"] = data["Mentions_Maisons_Contenu"] + data["Mentions_Maisons_Annexes"]
data["Mentions_Autres_Total"] = data["Mentions_Autres_Contenu"] + data["Mentions_Autres_Annexes"]
data["Mentions_Pairs_Total"] = data["Mentions_Pairs_Contenu"] + data["Mentions_Pairs_Annexes"]

data.head()


### Analyse des mentions

In [ ]:
# Regrouper par personne et par période pour calculer les statistiques
mentions_enquete_period = (
    data.groupby(["Nom_d_emprunt", "Periode"])
    .agg(
        Nombre_Total_Mentions=("Nombre_Mentions_Total", "sum"),
        Mentions_Auteurs_Total=("Mentions_Auteurs_Total", "sum"),
        Mentions_Maisons_Total=("Mentions_Maisons_Total", "sum"),
        Mentions_Pairs_Total=("Mentions_Pairs_Total", "sum"),
        Mentions_Autres_Total=("Mentions_Autres_Total", "sum"),
        Nombre_Mentions_Contenu=("Nombre_Mentions_Contenu", "sum"),
        Nombre_Mentions_Annexes=("Nombre_Mentions_Annexes", "sum"),
        Posts_Contenant_Mentions_Contenu=("Contient_Mentions_Contenu", "sum"),
        Posts_Contenant_Mentions_Annexes=("Contient_Mentions_Annexes", "sum"),
        Posts_Contenant_Mentions_Total=("Contient_Mention", "sum"),
        Posts_Contenant_Mentions_Pairs=("Contient_Mentions_Pairs_Total", "sum"),
        Posts_Contenant_Mentions_Auteurs=("Contient_Mentions_Auteurs_Total", "sum"),
        Posts_Contenant_Mentions_Autres=("Contient_Mentions_Autres_Total", "sum"),
        Posts_Contenant_Mentions_Maisons=("Contient_Mentions_Maisons_Total", "sum"),
        Nombre_de_Posts=("Date_Post", "count")  # Nombre total de posts
    )
    .reset_index()
)

In [ ]:
# Calculer les pourcentages
cols_mentions = [
    "Posts_Contenant_Mentions_Contenu",
    "Posts_Contenant_Mentions_Annexes",
    "Posts_Contenant_Mentions_Pairs",
    "Posts_Contenant_Mentions_Auteurs",
    "Posts_Contenant_Mentions_Autres",
    "Posts_Contenant_Mentions_Maisons"
]

# Calcul automatique des pourcentages
for col in cols_mentions:
    mentions_enquete_period[f"Pourcentage_{col}"] = (
        mentions_enquete_period[col] /
        mentions_enquete_period["Nombre_de_Posts"] * 100
    )

# Cas spécial : total contenu + annexes
mentions_enquete_period["Pourcentage_Posts_Contenant_Mentions_Total"] = (
    (mentions_enquete_period["Posts_Contenant_Mentions_Contenu"] +
     mentions_enquete_period["Posts_Contenant_Mentions_Annexes"]) /
    mentions_enquete_period["Nombre_de_Posts"] * 100
)
mentions_enquete_period

In [ ]:
mentions_period = (
    data.groupby(["Periode"])
    .agg(
        Nombre_Total_Mentions=("Nombre_Mentions_Total", "sum"),
        Mentions_Auteurs_Total=("Mentions_Auteurs_Total", "sum"),
        Mentions_Maisons_Total=("Mentions_Maisons_Total", "sum"),
        Mentions_Pairs_Total=("Mentions_Pairs_Total", "sum"),
        Mentions_Autres_Total=("Mentions_Autres_Total", "sum"),
        Nombre_Mentions_Contenu=("Nombre_Mentions_Contenu", "sum"),
        Nombre_Mentions_Annexes=("Nombre_Mentions_Annexes", "sum"),
        Posts_Contenant_Mentions_Contenu=("Contient_Mentions_Contenu", "sum"),
        Posts_Contenant_Mentions_Annexes=("Contient_Mentions_Annexes", "sum"),
        Posts_Contenant_Mentions_Total=("Contient_Mention", "sum"),
        Posts_Contenant_Mentions_Pairs=("Contient_Mentions_Pairs_Total", "sum"),
        Posts_Contenant_Mentions_Auteurs=("Contient_Mentions_Auteurs_Total", "sum"),
        Posts_Contenant_Mentions_Autres=("Contient_Mentions_Autres_Total", "sum"),
        Posts_Contenant_Mentions_Maisons=("Contient_Mentions_Maisons_Total", "sum"),
        Nombre_de_Posts=("Date_Post", "count")  # Nombre total de posts
    )
    .reset_index()
)

In [ ]:
# Calculer les pourcentages
cols_mentions = [
    "Posts_Contenant_Mentions_Contenu",
    "Posts_Contenant_Mentions_Annexes",
    "Posts_Contenant_Mentions_Pairs",
    "Posts_Contenant_Mentions_Auteurs",
    "Posts_Contenant_Mentions_Autres",
    "Posts_Contenant_Mentions_Maisons"
]

# Calcul automatique des pourcentages
for col in cols_mentions:
    mentions_period[f"Pourcentage_{col}"] = (
        mentions_period[col] /
        mentions_period["Nombre_de_Posts"] * 100
    )

# les totaux
mentions_period["Pourcentage_Posts_Contenant_Mentions_Total"] = (
    (mentions_period["Posts_Contenant_Mentions_Contenu"] +
     mentions_period["Posts_Contenant_Mentions_Annexes"]) /
    mentions_period["Nombre_de_Posts"] * 100
)
mentions_period

In [ ]:
mentions_enquete = (
    data.groupby(["Nom_d_emprunt"])
    .agg(
        Nombre_Total_Mentions=("Nombre_Mentions_Total", "sum"),
        Mentions_Auteurs_Total=("Mentions_Auteurs_Total", "sum"),
        Mentions_Maisons_Total=("Mentions_Maisons_Total", "sum"),
        Mentions_Pairs_Total=("Mentions_Pairs_Total", "sum"),
        Mentions_Autres_Total=("Mentions_Autres_Total", "sum"),
        Nombre_Mentions_Contenu=("Nombre_Mentions_Contenu", "sum"),
        Nombre_Mentions_Annexes=("Nombre_Mentions_Annexes", "sum"),
        Posts_Contenant_Mentions_Contenu=("Contient_Mentions_Contenu", "sum"),
        Posts_Contenant_Mentions_Annexes=("Contient_Mentions_Annexes", "sum"),
        Posts_Contenant_Mentions_Total=("Contient_Mention", "sum"),
        Posts_Contenant_Mentions_Pairs=("Contient_Mentions_Pairs_Total", "sum"),
        Posts_Contenant_Mentions_Auteurs=("Contient_Mentions_Auteurs_Total", "sum"),
        Posts_Contenant_Mentions_Autres=("Contient_Mentions_Autres_Total", "sum"),
        Posts_Contenant_Mentions_Maisons=("Contient_Mentions_Maisons_Total", "sum"),
        Nombre_de_Posts=("Date_Post", "count")  # Nombre total de posts
    )
    .reset_index()
)
# Calculer les pourcentages
cols_mentions = [
    "Posts_Contenant_Mentions_Contenu",
    "Posts_Contenant_Mentions_Annexes",
    "Posts_Contenant_Mentions_Pairs",
    "Posts_Contenant_Mentions_Auteurs",
    "Posts_Contenant_Mentions_Autres",
    "Posts_Contenant_Mentions_Maisons"
]

# Calcul automatique des pourcentages
for col in cols_mentions:
    mentions_enquete[f"Pourcentage_{col}"] = (
        mentions_enquete[col] /
        mentions_enquete["Nombre_de_Posts"] * 100
    )

# les totaux
mentions_enquete["Pourcentage_Posts_Contenant_Mentions_Total"] = (
    (mentions_enquete["Posts_Contenant_Mentions_Contenu"] +
     mentions_enquete["Posts_Contenant_Mentions_Annexes"]) /
    mentions_enquete["Nombre_de_Posts"] * 100
)
mentions_enquete

In [ ]:
mentions_enquete['type_carriere'] = mentions_enquete['Nom_d_emprunt'].apply(get_type_carriere)
# Regroupement par type_carriere avec somme pour les compteurs
mentions_carriere = mentions_enquete.groupby("type_carriere").agg({
    "Nombre_Total_Mentions": "sum",
    "Mentions_Auteurs_Total": "sum",
    "Mentions_Maisons_Total": "sum",
    "Mentions_Pairs_Total": "sum",
    "Mentions_Autres_Total": "sum",
    "Nombre_Mentions_Contenu": "sum",
    "Nombre_Mentions_Annexes": "sum",
    "Posts_Contenant_Mentions_Contenu": "sum",
    "Posts_Contenant_Mentions_Annexes": "sum",
    "Posts_Contenant_Mentions_Total": "sum",
    "Posts_Contenant_Mentions_Pairs": "sum",
    "Posts_Contenant_Mentions_Auteurs": "sum",
    "Posts_Contenant_Mentions_Autres": "sum",
    "Posts_Contenant_Mentions_Maisons": "sum",
    "Nombre_de_Posts": "sum",
})

# Calculer les pourcentages moyens pondérés par nombre de posts (optionnel)
# Par exemple : moyenne des pourcentages, pondérée par nombre de posts
for col in [
    "Pourcentage_Posts_Contenant_Mentions_Contenu",
    "Pourcentage_Posts_Contenant_Mentions_Annexes",
    "Pourcentage_Posts_Contenant_Mentions_Total"
]:
    mentions_carriere[col + "_moyenne_ponderee"] = (
        (mentions_enquete[col] * mentions_enquete["Nombre_de_Posts"]).groupby(mentions_enquete["type_carriere"]).sum()
        / mentions_enquete.groupby("type_carriere")["Nombre_de_Posts"].sum()
    )

mentions_carriere


In [ ]:
# Calcul des statistiques globales
stats_globales = {
    "Nombre_Total_Posts": len(data),
    "Nombre_Total_Mentions": data["Nombre_Mentions_Total"].sum(),
    "Mentions_Auteurs_Total": data["Mentions_Auteurs_Total"].sum(),
    "Mentions_Maisons_Total": data["Mentions_Maisons_Total"].sum(),
    "Mentions_Pairs_Total": data["Mentions_Pairs_Total"].sum(),
    "Mentions_Autres_Total": data["Mentions_Autres_Total"].sum(),
    "Pourcentage_Posts_Contenant_Mentions_": data["Contient_Mentions_Contenu"].sum() / len(data) * 100,
    "Pourcentage_Posts_Contenant_Mentions_Annexes": data["Contient_Mentions_Annexes"].sum() / len(data) * 100,
    "Pourcentage_Posts_Contenant_Mentions_Total": (data["Contient_Mention"].sum() / len(data)) * 100,
    "Posts_Contenant_Mentions_Contenu": data["Contient_Mentions_Contenu"].sum(),
    "Posts_Contenant_Mentions_Annexes": data["Contient_Mentions_Annexes"].sum(),
    "Posts_Contenant_Mentions_Total": data["Contient_Mention"].sum(),
    "Posts_Contenant_Mentions_Pairs": data["Contient_Mentions_Pairs_Total"].sum(),
    "Posts_Contenant_Mentions_Auteurs": data["Contient_Mentions_Auteurs_Total"].sum(),
    "Posts_Contenant_Mentions_Autres": data["Contient_Mentions_Autres_Total"].sum(),
    "Posts_Contenant_Mentions_Maisons": data["Contient_Mentions_Maisons_Total"].sum(),
}
    

# Afficher les statistiques globales
for key, value in stats_globales.items():
    print(f"{key}: {value}")

## Hashtag dans les posts

In [ ]:
# Analyse pour contenu et annexes
for zone, serie in [("Contenu", data["Contenu_Post"]), ("Annexes", data["Annexes_Post"])]:
    mentions = serie.apply(analyser_hashtags)
    for key in mentions.iloc[0].keys():  # récupère toutes les clés du dict
        data[f"{key}_{zone}"] = mentions.apply(lambda x: x[key])

# Calcul des totaux (hashtags contenu + annexes)
data["Nombre_Hashtags_Total"] = data["Nombre_Hashtags_Contenu"] + data["Nombre_Hashtags_Annexes"]
data["Contient_Hashtags"] = data["Contient_Hashtags_Contenu"] | data["Contient_Hashtags_Annexes"]
data.head()

### Analyse des hastags

In [ ]:
# Regrouper par personne et par période pour calculer les statistiques
hashtags_enquete_periode = (
    data.groupby(["Nom_d_emprunt", "Periode"])
    .agg(
        Nombre_Total_Hashtags=("Nombre_Hashtags_Total", "sum"),
        Nombre_Hashtags_Contenu=("Nombre_Hashtags_Contenu", "sum"),
        Nombre_Hashtags_Annexes=("Nombre_Hashtags_Annexes", "sum"),
        Nombre_de_Posts=("Date_Post", "count")  # Nombre total de posts
    )
    .reset_index()
)

# Calculer le pourcentage de posts contenant des hashtags
hashtags_enquete_periode["Pourcentage_Posts_Contenant_Hashtags_Contenu"] = (
    (data["Nombre_Hashtags_Contenu"] > 0).groupby([data["Nom_d_emprunt"], data["Periode"]]).sum() /
    hashtags_enquete_periode.set_index(["Nom_d_emprunt", "Periode"])["Nombre_de_Posts"] * 100
).values
hashtags_enquete_periode["Pourcentage_Posts_Contenant_Hashtags_Annexes"] = (
    (data["Nombre_Hashtags_Annexes"] > 0).groupby([data["Nom_d_emprunt"], data["Periode"]]).sum() /
    hashtags_enquete_periode.set_index(["Nom_d_emprunt", "Periode"])["Nombre_de_Posts"] * 100
).values
hashtags_enquete_periode["Pourcentage_Posts_Contenant_Hashtags_Total"] = (
    ((data["Nombre_Hashtags_Contenu"] > 0) | (data["Nombre_Hashtags_Annexes"] > 0)).groupby([data["Nom_d_emprunt"], data["Periode"]]).sum() /
    hashtags_enquete_periode.set_index(["Nom_d_emprunt", "Periode"])["Nombre_de_Posts"] * 100
).values

hashtags_enquete_periode

In [ ]:
hashtags_enquete = (
    data.groupby("Nom_d_emprunt")
    .agg(
        Nombre_Total_Hashtags=("Nombre_Hashtags_Total", "sum"),
        Nombre_Hashtags_Contenu=("Nombre_Hashtags_Contenu", "sum"),
        Nombre_Hashtags_Annexes=("Nombre_Hashtags_Annexes", "sum"),
        Nombre_de_Posts=("Date_Post", "count")  # Total de posts par personne
    )
    .reset_index()
)

# Calcul du pourcentage de posts avec hashtags pour chaque enquêté
hashtags_enquete["Pourcentage_Posts_Contenant_Hashtags_Contenu"] = (
    (data["Nombre_Hashtags_Contenu"] > 0).groupby(data["Nom_d_emprunt"]).sum() /
    hashtags_enquete.set_index("Nom_d_emprunt")["Nombre_de_Posts"] * 100
).values
hashtags_enquete["Pourcentage_Posts_Contenant_Hashtags_Annexes"] = (
    (data["Nombre_Hashtags_Annexes"] > 0).groupby(data["Nom_d_emprunt"]).sum() /
    hashtags_enquete.set_index("Nom_d_emprunt")["Nombre_de_Posts"] * 100
).values
hashtags_enquete["Pourcentage_Posts_Contenant_Hashtags_Total"] = (
    ((data["Nombre_Hashtags_Contenu"] > 0) | (data["Nombre_Hashtags_Annexes"] > 0))
    .groupby(data["Nom_d_emprunt"]).sum() /
    hashtags_enquete.set_index("Nom_d_emprunt")["Nombre_de_Posts"] * 100
).values

hashtags_enquete


In [ ]:
hashtags_enquete['type_carriere'] = hashtags_enquete['Nom_d_emprunt'].apply(get_type_carriere)

In [ ]:
# Analyse des hastags par type de carrière
hashtags_carriere = (
    hashtags_enquete
    .groupby("type_carriere")
    .agg(
        Nombre_Total_Hashtags=("Nombre_Total_Hashtags", "sum"),
        Nombre_Hashtags_Contenu=("Nombre_Hashtags_Contenu", "sum"),
        Nombre_Hashtags_Annexes=("Nombre_Hashtags_Annexes", "sum"),
        Nombre_de_Posts=("Nombre_de_Posts", "sum"),
        Pourcentage_Moyen_Posts_Contenant_Hashtags_Contenu=("Pourcentage_Posts_Contenant_Hashtags_Contenu", "mean"),
        Pourcentage_Moyen_Posts_Contenant_Hashtags_Annexes=("Pourcentage_Posts_Contenant_Hashtags_Annexes", "mean"),
        Pourcentage_Moyen_Posts_Contenant_Hashtags_Total=("Pourcentage_Posts_Contenant_Hashtags_Total", "mean"),
    )
    .reset_index()
)

hashtags_carriere = hashtags_carriere.set_index("type_carriere")
hashtags_carriere


In [ ]:
# Analyse des hashtags par période
hashtags_periode = (
    data.groupby([ "Periode"])
    .agg(
        Nombre_Total_Hashtags=("Nombre_Hashtags_Total", "sum"),
        Nombre_Hashtags_Contenu=("Nombre_Hashtags_Contenu", "sum"),
        Nombre_Hashtags_Annexes=("Nombre_Hashtags_Annexes", "sum"),
        Nombre_de_Posts=("Date_Post", "count")  
    )
    .reset_index()
)

# Calculer le pourcentage de posts contenant des hashtags
hashtags_periode["Pourcentage_Posts_Contenant_Hashtags_Contenu"] = (
    (data["Nombre_Hashtags_Contenu"] > 0).groupby([data["Periode"]]).sum() /
    hashtags_periode.set_index(["Periode"])["Nombre_de_Posts"] * 100
).values
hashtags_periode["Pourcentage_Posts_Contenant_Hashtags_Annexes"] = (
    (data["Nombre_Hashtags_Annexes"] > 0).groupby([data["Periode"]]).sum() /
    hashtags_periode.set_index(["Periode"])["Nombre_de_Posts"] * 100
).values
hashtags_periode["Pourcentage_Posts_Contenant_Hashtags_Total"] = (
    ((data["Nombre_Hashtags_Contenu"] > 0) | (data["Nombre_Hashtags_Annexes"] > 0)).groupby([data["Periode"]]).sum() /
    hashtags_periode.set_index(["Periode"])["Nombre_de_Posts"] * 100
).values

hashtags_periode

In [ ]:
# Calcul des statistiques globales à propos des hashtags
hashtags_globales = {
    "Nombre_Total_Posts": len(data),
    "Nombre_Total_Hashtags": data["Nombre_Hashtags_Total"].sum(),
    "Nombre_Hashtags_Contenu": data["Nombre_Hashtags_Contenu"].sum(),
    "Nombre_Hashtags_Annexes": data["Nombre_Hashtags_Annexes"].sum(),
    "Nombre_Hashtags_Total": data["Nombre_Hashtags_Total"].sum(),
    "Pourcentage_Posts_Contenant_Hashtags_Contenu": data["Contient_Hashtags_Contenu"].sum() / len(data) * 100,
    "Pourcentage_Posts_Contenant_Hashtags_Annexes": data["Contient_Hashtags_Annexes"].sum() / len(data) * 100,
    "Pourcentage_Posts_Contenant_Hashtags_Total": (data["Contient_Hashtags"].sum() / len(data)) * 100,
    "Posts_Contenant_Hashtags_Contenu": data["Contient_Hashtags_Contenu"].sum(),
    "Posts_Contenant_Hashtags_Annexes": data["Contient_Hashtags_Annexes"].sum(),   
}
    
for key, value in hashtags_globales.items():
    print(f"{key}: {value}")

## Analyses des images 

In [ ]:
# Liste des colonnes binaires
colonnes_binaires = ['Plusieurs_Images', 'Personne', 'PostProduction', 'Mots']
images_globales = {}

for col in colonnes_binaires:
    images_globales[col] = {
        'Présence': data[col].sum(),
        'Absence': data[col].count() - data[col].sum(),
        'Pourcentage_Présence': data[col].mean() * 100,
        'Total': data[col].count()
    }

images_globales = pd.DataFrame(images_globales)
images_globales

# lister les posts sur lesquels il n'y a pas ces informations pour vérifier les erreurs potentielles
#posts_sans_info = data[data[colonnes_binaires].isnull().any(axis=1)][['ID_Post', 'Nom_d_emprunt', 'Periode', 'Date_Post'] + colonnes_binaires]
#posts_sans_info

In [ ]:
# Analyse par personne des images
images_enquete = data.groupby('Nom_d_emprunt')[colonnes_binaires].sum()
images_enquete['Total_Posts'] = data.groupby('Nom_d_emprunt').size()
images_enquete = images_enquete.div(images_enquete['Total_Posts'], axis=0) * 100

images_enquete

In [ ]:
# enlever les posts dont les images ne sont plus disponibles lors de la collecte des données
colonnes_image = ["Personne", "PostProduction", "Mots", "Plusieurs_Images"]
data_valable = data.dropna(subset=colonnes_image, how='all')

# Calculer les totaux des colonnes binaires, par période, sur ces données filtrées
images_periode = data_valable.groupby('Periode')[colonnes_binaires].sum()
images_periode['Total_Posts'] = data.groupby('Periode').size()

images_periode

In [ ]:
# Analyse par carrière des images
images_carriere = data.groupby('type_carriere')[colonnes_binaires].sum()
images_carriere['Total_Posts'] = data.groupby('type_carriere').size()
images_carriere = images_carriere.div(images_carriere['Total_Posts'], axis=0) * 100

images_carriere

In [ ]:
# Analyse croisée par personne et période
images_enquete_periode = data.groupby(['Nom_d_emprunt', 'Periode'])[colonnes_binaires].sum()
images_enquete_periode['Total_Posts'] = data.groupby(['Nom_d_emprunt', 'Periode']).size()
images_enquete_periode = images_enquete_periode.div(images_enquete_periode['Total_Posts'], axis=0) * 100
images_enquete_periode


## Analyse de la nature des posts 

In [ ]:
# Analyse globale
nature_globales = data[['N_Ecrit', 'N_Video', 'N_Audio']].sum().to_frame(name='Total')
nature_globales['Pourcentage'] = (nature_globales['Total'] /596) * 100
nature_globales['Total_Posts'] = len(data)

nature_globales

In [ ]:
# Analyse par enquete
nature_enquete = data.groupby('Nom_d_emprunt')[['N_Ecrit', 'N_Video', 'N_Audio']].sum()
nature_enquete['Total_Posts'] = data.groupby('Nom_d_emprunt').size()
nature_enquete

In [ ]:
# Analyse par période
nature_periode = data.groupby('Periode')[['N_Ecrit', 'N_Video', 'N_Audio']].sum()
nature_periode['Total_Posts'] = data.groupby('Periode').size()
nature_periode = nature_periode.div(nature_periode['Total_Posts'], axis=0) * 100
nature_periode

In [ ]:
# Analyse croisée par personne et période
nature_enquete_periode = data.groupby(['Nom_d_emprunt', 'Periode'])[['N_Ecrit', 'N_Video', 'N_Audio']].sum()
nature_enquete_periode['Total_Posts'] = data.groupby(['Nom_d_emprunt', 'Periode']).size()
nature_enquete_periode = nature_enquete_periode.div(nature_enquete_periode['Total_Posts'], axis=0) * 100
nature_enquete_periode

In [ ]:
# Analyse par carriere
nature_carriere = data.groupby('type_carriere')[['N_Ecrit', 'N_Video', 'N_Audio']].sum()
nature_carriere['Total_Posts'] = data.groupby('type_carriere').size()
nature_carriere = nature_carriere.div(nature_carriere['Total_Posts'], axis=0) * 100

nature_carriere

# Tableaux pour le clustering

In [ ]:

supra_df = frequence.merge(enquete_longueur, on=["Nom_d_emprunt", "Periode"], how="left")
supra_df = supra_df.merge(mentions_enquete_period.drop(columns = "Nombre_de_Posts"), on=["Nom_d_emprunt", "Periode"], how="left")
supra_df = supra_df.merge(hashtags_enquete_periode, on=["Nom_d_emprunt", "Periode"], how="left")
supra_df = supra_df.merge(images_enquete_periode, on=["Nom_d_emprunt", "Periode"], how="left")
supra_df = supra_df.merge(nature_enquete_periode, on=["Nom_d_emprunt", "Periode"], how="left")

supra_df

In [ ]:
# Afficher la liste des noms de colonnes de merged_df
supra_df.columns.tolist()

On visualise les colonnes de notre df et on enleve celles qui ne sont pas nécessaires pour le clustering

In [ ]:
supra_df.drop(columns=[ 'Carrière',
 'Intervalle_Mediane',
 'Longueur_Minimale',
 'Longueur_Maximale',
 'Ecart_Type_Longueur',
 'Longueur_Mediane',
 "Nombres d'Annexes",
  'Intervalle_Minimum',
 'Intervalle_Maximum','Total_Posts_y','Total_Posts_x', 'Pourcentage_Posts_Contenant_Hashtags_Contenu',
 'Pourcentage_Posts_Contenant_Hashtags_Annexes', 'Nombre_Hashtags_Contenu',
 'Nombre_Hashtags_Annexes', 'Pourcentage_Posts_Contenant_Mentions_Contenu',
 'Pourcentage_Posts_Contenant_Mentions_Annexes', 'Nombre_Mentions_Contenu',
 'Nombre_Mentions_Annexes',
 'Posts_Contenant_Mentions_Contenu',
 'Posts_Contenant_Mentions_Annexes',
 'Nombre_de_Posts_y','Nombre_de_Posts_x',  'Mentions_Auteurs_Total',
 'Mentions_Maisons_Total',
 'Mentions_Autres_Total',
 'Nombre_Mentions_Contenu',
 'Nombre_Mentions_Annexes','Posts_Contenant_Mentions_Total',
 'Posts_Contenant_Mentions_Pairs',
 'Posts_Contenant_Mentions_Auteurs',
 'Posts_Contenant_Mentions_Autres',
 'Posts_Contenant_Mentions_Maisons','Pourcentage_Posts_Contenant_Mentions_Autres','Mentions_Pairs_Total'], inplace=True)

In [ ]:
# Sauvegarder le DataFrame final pour le clustering
#merged_df.to_excel("Clustering_Data.xlsx", index=False)
supra_df.columns.tolist()

In [ ]:
supra_df
# On obtient le DataFrame final avec les colonnes nécessaires pour le clustering des couples enquêtés et périodes.

In [ ]:
# Pour obtenir le dataframe nécesaire au clustering sur les enquetés, on va faire des sommes et des moyennes pondérées sur les indicateurs statistiques. 
df = supra_df.copy()

colonnes_a_sommer = [  'Nombre_de_Posts', 'Nombre_Total_Hashtags', 'Nombre_Total_Mentions',
  ]

colonnes_a_pond_moy = [
'Plusieurs_Images',
 'Personne',
 'PostProduction',
 'Mots',
 'N_Ecrit',
 'N_Video',
 'N_Audio',  'Pourcentage_Posts_Contenant_Hashtags_Total', 'Pourcentage_Posts_Contenant_Mentions_Total',  'Longueur_Moyenne',
 'Pourcentage_Annexes',
 'Intervalle_Moyen',
 'Ecart_Type_Intervalle', 
 'Pourcentage_Posts_Contenant_Mentions_Pairs',
 'Pourcentage_Posts_Contenant_Mentions_Auteurs',
 'Pourcentage_Posts_Contenant_Mentions_Maisons',
]

somme = df.groupby("Nom_d_emprunt")[colonnes_a_sommer].sum().reset_index()

for col in colonnes_a_pond_moy:
    df[col + "_pond"] = df[col] * df["Nombre_de_Posts"]

ponds = (
    df.groupby("Nom_d_emprunt")[[col + "_pond" for col in colonnes_a_pond_moy] + ["Nombre_de_Posts"]]
    .sum()
    .reset_index()
)

for col in colonnes_a_pond_moy:
    ponds[col] = ponds[col + "_pond"] / ponds["Nombre_de_Posts"]

ponds = ponds[["Nom_d_emprunt"] + colonnes_a_pond_moy]

df_global = pd.merge(somme, ponds, on="Nom_d_emprunt")

df_global.head()
#df_global.to_excel("Clustering_Global.xlsx", index=False)
